# Informed Search: Admissibility

This notebook will investigate the admissibility of priority list heuristics. 

In order for informed methods like A* search to be optimal, it must be admissible: $h(n) \leq h^{*}(n)$. In other words, it must be an optimistic estimate of the path cost. 

One way to design a heuristic satisfying this constraint is to design a heuristic based on a priority list. Priority list methods commit generators in order of marginal cost. Often the operating constraints are ignored initially, and then some heuristic methods are used to fix the resulting schedule. 

In `ts4uc` we have designed a simple heuristic that commits generators in priority list order, ignoring constraints, then calculates the economic dispatch by simply using the cost per MWh when operating at full capacity. This is very fast to compute and mainly serves to reflect the fact that nodes that are closer to the end of the day will have lower 'cost-to-go' (the cost to the end of the day) then earlier nodes. Importantly, this heuristic does not distinguish between heuristics at the same generation. 

In [238]:
from rl4uc.environment import make_env

from ts4uc.tree_search import informed_search
from ts4uc.tree_search.scenarios import get_net_demand_scenarios
from ts4uc.tree_search.algos import *
from ts4uc.tree_search import node
from ts4uc.agents.ac_agent import ACAgent
from ts4uc import helpers

import numpy as np 
import pandas as pd 
import torch
import json

### User Inputs

In [239]:
ENV_PARAMS_FN = '../data/day_ahead/5gen/30min/env_params.json'
TEST_DATA_BASE_FN = '../data/day_ahead/5gen/30min/profile_{}.csv'
HORIZON = 2
BRANCHING_THRESHOLD = 0.05
HEURISTIC_METHOD = 'priority_list'
SEED = 1 
NUM_SCENARIOS = 100
TEST_SAMPLE_SEED = 999
NUM_SAMPLES = 1000
NUM_PERIODS=48

We will redefine the `solve_day_ahead()` function in order to keep track of the heuristic cost and return this at the end of the search. 

We will also redefine our function for testing schedules to keep track of the period-level operating costs.

In [240]:
def solve_day_ahead(env, 
                    horizon, 
                    net_demand_scenarios,
                    tree_search_func=uniform_cost_search, 
                    **params):
    """
    Solve a day rooted at env. 
    
    Return the schedule and the number of branches at the root for each time period. 
    """
    env.reset()
    final_schedule = np.zeros((env.episode_length, env.num_gen))

    root = node.Node(env=env,
            parent=None,
            action=None,
            step_cost=0,
            path_cost=0)
    
    heuristic_costs = []

    for t in range(env.episode_length):
                
        s = time.time()
        terminal_timestep = min(env.episode_timestep + horizon, env.episode_length-1)
        
        # RTA* heuristic estimate
        h = root.state.episode_length - root.state.episode_timestep -1 
        c = informed_search.heuristic(root, h, params.get('heuristic_method'))
        heuristic_costs.append(c)
        print("Heuristic cost: {}, lookahead: {}".format(c, h))
        
        path, cost = tree_search_func(root, 
                                      terminal_timestep, 
                                      net_demand_scenarios,
                                      **params)
        a_best = path[0]

        print(f"Period {env.episode_timestep+1}", np.array(a_best, dtype=int), round(cost, 2), round(time.time()-s, 2))
        final_schedule[t, :] = a_best
        env.step(a_best, deterministic=True)
        
        root = root.children[a_best.tobytes()]
        root.parent, root.path_cost = None, 0

        gc.collect()
            
    return final_schedule, heuristic_costs

def test_schedule(env, schedule, seed=999, num_samples=1000, deterministic=False):
    costs = np.zeros((num_samples, env.episode_length))
    np.random.seed(seed)
    for i in range(num_samples):
        env.reset()
        total_reward = 0 
        for action in schedule:
            action = np.where(np.array(action)>0, 1, 0)
            obs,reward,done = env.step(action, deterministic)
            costs[i, env.episode_timestep] = -reward
    return costs


In [241]:
def calc_heuristic_preds(test_data_fn):
    np.random.seed(SEED)
    torch.manual_seed(SEED)

    # Load parameters
    env_params = json.load(open(ENV_PARAMS_FN))

    # Load profile 
    profile_df = pd.read_csv(test_data_fn)
    profile_df = profile_df[:NUM_PERIODS]

    params = {'horizon': HORIZON,
              'branching_threshold': BRANCHING_THRESHOLD,
              'heuristic_method': HEURISTIC_METHOD}

    # Init env
    env = make_env(mode='test', profiles_df=profile_df, **env_params)

    # Load policy
    policy = None

    # Generate scenarios for demand and wind errors
    scenarios = get_net_demand_scenarios(profile_df, env, NUM_SCENARIOS)

    schedule_result, heuristic_costs = solve_day_ahead(env=env, 
                                                       net_demand_scenarios=scenarios, 
                                                       tree_search_func=rta_star,
                                                       policy=policy,
                                                       **params)

    TEST_SAMPLE_SEED=999
    test_costs = test_schedule(env, schedule_result, TEST_SAMPLE_SEED, NUM_SAMPLES)

    pred = np.array(heuristic_costs[:])

    actual = []
    for i in range(NUM_PERIODS):
        actual.append(np.cumsum(test_costs, axis=1).T[-(i+1)].mean())
    actual = np.array(actual)
    
    return pd.DataFrame({'pred': pred,
                         'actual': actual})

In [ ]:
df_pred_ls = []
for date in ['2019-05-22', '2017-04-07', '2018-05-30']:
    test_data_fn = TEST_DATA_BASE_FN.format(date)
    df_pred_ls.append(calc_heuristic_preds(test_data_fn))
df_pred = pd.concat(df_pred_ls)

Heuristic cost: 195969.12427138275, lookahead: 48
Period 0 [1 0 0 0 0] 9866.47 2.76
Heuristic cost: 192109.48562491368, lookahead: 47
Period 1 [1 0 0 0 0] 9694.44 2.26
Heuristic cost: 188408.28561736873, lookahead: 46
Period 2 [1 0 0 0 0] 9663.18 2.28
Heuristic cost: 184764.36593256242, lookahead: 45
Period 3 [1 0 0 0 0] 9720.18 2.21
Heuristic cost: 181103.12136163432, lookahead: 44
Period 4 [1 0 0 0 0] 9834.8 2.22
Heuristic cost: 177389.5801224799, lookahead: 43
Period 5 [1 0 0 0 1] 9814.82 2.24
Heuristic cost: 173720.27545117322, lookahead: 42
Period 6 [1 0 0 0 1] 9705.69 1.54
Heuristic cost: 170113.75452276116, lookahead: 41
Period 7 [1 0 0 0 1] 9801.99 2.15
Heuristic cost: 166525.10119210818, lookahead: 40
Period 8 [1 0 0 0 1] 9950.33 2.18
Heuristic cost: 162790.37613715135, lookahead: 39
Period 9 [1 0 0 0 1] 9850.73 1.87
Heuristic cost: 159024.3227610058, lookahead: 38
Period 10 [1 0 0 0 1] 9865.11 2.13
Heuristic cost: 155345.23445804472, lookahead: 37
Period 11 [1 0 0 0 1] 10144.

In [ ]:
import matplotlib.pyplot as plt

xmin = np.min(df_pred.pred)
xmax = np.max(df_pred.actual)
xmin_to_xmax = np.linspace(xmin, xmax)

fig, ax = plt.subplots(dpi=125)
ax.plot(xmin_to_xmax, xmin_to_xmax, c='r')
ax.scatter(df_pred.actual, df_pred.pred, s=1)
ax.set_xlabel('actual')
ax.set_ylabel('predicted')
plt.show()